In [30]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
dataset = load_dataset('trec')
#display(dataset)
dataset = dataset.remove_columns("label-fine")
#display(dataset)
dataset = dataset.rename_column("label-coarse", "label")
display(dataset)


train_validation_ds = dataset['train'].shuffle(seed=42).select([i for i in list(range(5000))])
train_ds = Dataset.from_dict(train_validation_ds[0:4500])
validation_ds = Dataset.from_dict(train_validation_ds[4500:5000])
test_ds = dataset['test'].shuffle(seed=42).select([i for i in list(range(500))])




display(train_ds)
train_df = pd.DataFrame(train_ds)
display(train_df)


Using the latest cached version of the module from C:\Users\DELL\.cache\huggingface\modules\datasets_modules\datasets\trec\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9 (last modified on Sun Sep 11 21:40:15 2022) since it couldn't be found locally at trec., or remotely on the Hugging Face Hub.
Using custom data configuration default
Reusing dataset trec (C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 668.36it/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 500
    })
})

Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9\cache-bc761497303cb1ab.arrow
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9\cache-d432b7eeb4042bcd.arrow


Dataset({
    features: ['label', 'text'],
    num_rows: 4500
})

,label,text
0,0,How can I transport files from one computer to...
1,3,Who are the two sons of Ozzie and Harriet Nels...
2,2,What does pH stand for ?
3,4,How many people does Honda employ in the U.S. ?
4,1,What newspaper returned a Pulitzer Prize for t...
...,...,...
4495,1,What does Final Four refer to in the sports wo...
4496,3,What is Nicholas Cage 's occupation ?
4497,1,"In order from the top , the four stripes on a ..."
4498,3,What kind of business is 7-Eleven ?


In [31]:
#wordnet part
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import wordnet


from senticnet.senticnet import SenticNet
import pandas as pd

import spacy
from spacy import displacy
from lemminflect import getInflection

In [32]:
#creating antonyms list using wordnet
def antonyms_list_creation(target_word):
    antonyms_list = []
    try:
        for ann in wordnet.synsets(target_word):
            for lm in ann.lemmas():
                if lm.antonyms():
                    antonyms_list.append(lm.antonyms()[0].name())
        #print (set(antonyms_list))
        antonyms_list = list(set(antonyms_list))
        #transform _ to space
        antonyms_list = [s.replace('_', ' ') for s in antonyms_list]


        antonyms_list
        antonyms_list = [s for s in antonyms_list if ' ' not in s]
        
    except Exception as e:
        pass
    
    return antonyms_list

In [33]:
from textattack.transformations import WordSwap
class Antonyms_WordSwap(WordSwap):
    """ Transforms an input by replacing any word with 'banana'.
    """
    
    # We don't need a constructor, since our class doesn't require any parameters.

    def _get_replacement_words(self, word):
        antonyms_list = antonyms_list_creation(word)
        
        return antonyms_list

In [34]:
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.grammaticality.part_of_speech import PartOfSpeech
from textattack.constraints.semantics.bert_score import BERTScore

from textattack.augmentation import Augmenter


transformation = Antonyms_WordSwap()

# Set up constraints
constraints = [RepeatModification(), StopwordModification(),
               PartOfSpeech(tagger_type='stanza', tagset='universal', allow_verb_noun_swap=False, compare_against_original=True, language_nltk='eng', language_stanza='en')
            ]
# Create augmenter with specified parameters
augmenter = Augmenter(transformation=transformation, constraints=constraints, pct_words_to_swap=0.1, transformations_per_example=1)

Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |

Use device: gpu
Loading: tokenize
Loading: pos
Done loading processors!


In [35]:
# Augment!
# additional parameters can be modified if not during initiation




def results_list_creation(s):
    try:
        results = augmenter.augment(s)
        
    
    except Exception as e:
        results = ['Error occured in text generation .']
    
    return results

import time
from multiprocessing import Pool
from concurrent import futures

time_sta = time.perf_counter()
results_list = []

#print(list(train_df['text']))

with futures.ThreadPoolExecutor() as executor:
    results_list_itera = executor.map(results_list_creation, list(train_df['text'])) 

for results in results_list_itera:
    results_list.append(results)



time_end = time.perf_counter()
tim = time_end - time_sta

In [36]:


print(tim/3600)




0.03673307413888889


In [37]:
display(results_list)

[['How can I disenchant files from one computer to another ?'],
 ['Who are the two girl of Ozzie and Harriet Nelson ?'],
 ['What does pH yield for ?'],
 ['How few people does Honda employ in the U.S. ?'],
 ["What newspaper returned a Pulitzer Prize for the fraudulent story Jimmy 's World ?"],
 ['What two cities remarkably mark the extremes of English Channel swims ?'],
 ['Who was the mother of the woman who would not be king , the duke of Windsor ?'],
 ['What chocolate company take you a one-pound kiss ?'],
 ['What is the temperature for cooking ?'],
 ['Name the Islamic counterpart to the Red uncross .'],
 ['What are my legal left in an automobile repossession in California ?'],
 ['Where was helium last discovered , hence its name ?'],
 ["Whom does Uncle Duke 's boy friend , Honey , room with ?"],
 ['Who invented television ?'],
 ['What James Michener book is subtitled Spanish Travels and Reflections ?'],
 ['What does laser sit for ?'],
 ['What is the Homelite Inc. home page ?'],
 ["Wh

In [38]:
import itertools

original_sentences_list = []

for i in range(len(train_df)):
    number_of_generated_sentences_per_original_sentence = len(results_list[i])
    original_sentences = [list(train_df['text'])[i] for j in range(number_of_generated_sentences_per_original_sentence)]
    original_sentences_list.append(original_sentences)
display(original_sentences_list)

original_sentences_list_1d = list(itertools.chain.from_iterable(original_sentences_list))
results_list_1d = list(itertools.chain.from_iterable(results_list))


[['How can I transport files from one computer to another ?'],
 ['Who are the two sons of Ozzie and Harriet Nelson ?'],
 ['What does pH stand for ?'],
 ['How many people does Honda employ in the U.S. ?'],
 ["What newspaper returned a Pulitzer Prize for the fraudulent story Jimmy 's World ?"],
 ['What two cities usually mark the extremes of English Channel swims ?'],
 ['Who was the mother of the man who would not be king , the duke of Windsor ?'],
 ['What chocolate company gives you a one-pound kiss ?'],
 ['What is the temperature for cooking ?'],
 ['Name the Islamic counterpart to the Red Cross .'],
 ['What are my legal rights in an automobile repossession in California ?'],
 ['Where was helium first discovered , hence its name ?'],
 ["Whom does Uncle Duke 's girl friend , Honey , room with ?"],
 ['Who invented television ?'],
 ['What James Michener book is subtitled Spanish Travels and Reflections ?'],
 ['What does laser stand for ?'],
 ['What is the Homelite Inc. home page ?'],
 ["Wh

In [39]:
import pandas as pd 
augmented_data = {'original_text':original_sentences_list_1d,'perturbed_text':results_list_1d}
augmented_data_df = pd.DataFrame(augmented_data)
display(augmented_data_df)


,original_text,perturbed_text
0,How can I transport files from one computer to...,How can I disenchant files from one computer t...
1,Who are the two sons of Ozzie and Harriet Nels...,Who are the two girl of Ozzie and Harriet Nels...
2,What does pH stand for ?,What does pH yield for ?
3,How many people does Honda employ in the U.S. ?,How few people does Honda employ in the U.S. ?
4,What newspaper returned a Pulitzer Prize for t...,What newspaper returned a Pulitzer Prize for t...
...,...,...
4495,What does Final Four refer to in the sports wo...,What does Final Four refer to in the sports wo...
4496,What is Nicholas Cage 's occupation ?,What is Nicholas Cage 's occupation ?
4497,"In order from the top , the four stripes on a ...","In disorder from the top , the four stripes on..."
4498,What kind of business is 7-Eleven ?,What kind of business is 7-Eleven ?


In [40]:
#remove augmented_data_df['original_text'] == augmented_data_df['perturbed_text']
target = augmented_data_df.index[augmented_data_df['original_text'] == augmented_data_df['perturbed_text']]
augmented_data_df = augmented_data_df.drop(target)
augmented_data_df = augmented_data_df.reset_index(drop=True)
display(augmented_data_df)

,original_text,perturbed_text
0,How can I transport files from one computer to...,How can I disenchant files from one computer t...
1,Who are the two sons of Ozzie and Harriet Nels...,Who are the two girl of Ozzie and Harriet Nels...
2,What does pH stand for ?,What does pH yield for ?
3,How many people does Honda employ in the U.S. ?,How few people does Honda employ in the U.S. ?
4,What two cities usually mark the extremes of E...,What two cities remarkably mark the extremes o...
...,...,...
2870,What government department did The Untouchable...,What government department did The Untouchable...
2871,What is the average age of a member of the tea...,What is the average age of a nonmember of the ...
2872,How fast must a spacecraft travel to escape Ea...,How fast must a spacecraft travel to escape Ea...
2873,"In order from the top , the four stripes on a ...","In disorder from the top , the four stripes on..."


In [41]:
#save to csv file
augmented_data_df.to_csv( r'C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\augumented_dataset\Antonym\with_pos\allow_verb_noun_swap=False\run3\dataset.csv')